# Fix Coverage Errors from using y_binary to y_score

In [16]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from collections import namedtuple
import random
import cPickle as pickle

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info

from thesis.utils.metrics import *

2017-01-17 20:14:49,192 : INFO : Pattern library is not installed, lemmatization won't be available.
Using gpu device 0: Tesla K40m (CNMeM is disabled, cuDNN 5105)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
2017-01-17 20:15:25,231 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
CLASSIFIER_FILE = '{}_classifier.pkl'
VALIDATION_METRICS_FILENAME= '{}_validation_metrics.pkl'
TRAINING_METRICS_FILENAME = '{}_training_metrics.pkl'

In [6]:
root_location = "/mnt/data2/shalaby/"
exports_location = root_location + "exported_data/"
svm_location = root_location + "benchmarking_svm/"


training_file = root_location + "docs_output.json"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

#### Load Classification Objects

In [7]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))

CPU times: user 20.2 s, sys: 6.02 s, total: 26.2 s
Wall time: 26.2 s


In [8]:
import re
def get_subdirectories(d):
    #return filter(os.path.isdir, [f for f in os.listdir(d)])
    return [f for f in os.listdir(d) if os.path.isdir(os.path.join(d,f))]
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

## Fix Coverage Error for Benchmarking

In [ ]:
classifications = sections
classifications_type = "sections"

In [ ]:
%%time
# data_types = ["bm25", "tf_idf", "sublinear_tf_idf", "sublinear_tf", "tf"]
data_types = ["tf_idf", "sublinear_tf_idf", "sublinear_tf", "tf"]
# data_types = ["tf"]
for data_type in data_types:
    preprocessor = data_type
    info("=============== {} Being Evaluated ================".format(data_type))
    
    data_training_location = exports_location + "{}_training_sparse_data.pkl".format(data_type)
    data_training_docids_location = exports_location + "{}_training_sparse_docids.pkl".format(data_type)
    data_validation_location = exports_location + "{}_validation_sparse_data.pkl".format(data_type)
    data_validation_docids_location = exports_location + "{}_validation_sparse_docids.pkl".format(data_type)
    
#     # Get the training data
#     info('Getting Training Data')
#     %time X = pickle.load(open(data_training_location, "r"))
#     training_data_docids = pickle.load(open(data_training_docids_location, "r"))
#     %time y = get_label_data(classifications, training_data_docids, doc_classification_map)

#     print y
#     print y.shape
    
    # Get the validation data
    info('Getting Valdiation Data')
    %time Xv = pickle.load(open(data_validation_location,'r'))
    validation_data_docids = pickle.load(open(data_validation_docids_location, "r"))
    %time yv = get_label_data(classifications, validation_data_docids, doc_classification_map)
    
    print yv
    print yv.shape
        
    for classifier in natural_sort(get_subdirectories(svm_location)):
        print classifier
        classifier_path = os.path.join(svm_location, classifier)
        preprocessor_path = os.path.join(classifier_path, preprocessor)

        if not os.path.exists(preprocessor_path):
            info("{} doesn't exist, skipping".format(preprocessor_path)) 
            continue
            
        info('Loading Classifier')
        clf = pickle.load(open(os.path.join(preprocessor_path, CLASSIFIER_FILE.format(classifications_type)), 'r'))
        
#         # Training Metrics
#         info('Evaluating on Training Data')
#         %time yp = clf.predict(X)
#         %time yp_score = clf.decision_function(X)
#         print yp
#         info('Calculating training metrics')
#         training_metrics = get_metrics(y, yp_score, yp)
#         print "** Training Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
#             training_metrics['coverage_error'], training_metrics['average_num_of_labels'], 
#             training_metrics['top_1'], training_metrics['top_3'], training_metrics['top_5'], 
#             training_metrics['f1_micro'], training_metrics['f1_macro'], training_metrics['total_positive'])

        # Validation Metrics
        info('Evaluating on Validation Data')
        %time yvp = clf.predict(Xv)
        %time yvp_score = clf.decision_function(Xv)
        print yvp
        info('Calculating validation metrics')
        validation_metrics = get_metrics(yv, yvp_score, yvp)
        print "** Validation Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
            validation_metrics['coverage_error'], validation_metrics['average_num_of_labels'], 
            validation_metrics['top_1'], validation_metrics['top_3'], validation_metrics['top_5'], 
            validation_metrics['f1_micro'], validation_metrics['f1_macro'], validation_metrics['total_positive'])

        info('Dumping the Metrics')
#         pickle.dump(training_metrics, open(os.path.join(preprocessor_path, TRAINING_METRICS_FILENAME.format(classifications_type)), "w"))
        pickle.dump(validation_metrics, open(os.path.join(preprocessor_path, VALIDATION_METRICS_FILENAME.format(classifications_type)), "w"))
        

## Fix Coverage Error for Doc2vec

In [10]:
from gensim.models.doc2vec import Doc2Vec

2017-01-17 00:49:47,865 : INFO : Pattern library is not installed, lemmatization won't be available.
2017-01-17 00:49:47,903 : INFO : Could not import Theano, will use standard float for default ShardedCorpus dtype.
2017-01-17 00:49:48,126 : INFO : 'pattern' package not found; tag filters are not available for English


In [9]:
ROOT_RESULTS_LOCATION = '/mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/'

MODEL_PREFIX = "model"
VALIDATION_MATRIX = "validation_matrix.pkl"
CLASSIFIER = "classifier.pkl"

preprocessed_location = root_location + "preprocessed_data/"

validation_preprocessed_files_prefix = preprocessed_location + "validation_docs_merged_data_preprocessed-"
validation_preprocessed_docids_files_prefix = preprocessed_location + "validation_docs_merged_docids_preprocessed-"

In [10]:
NUM_CORES = 16
SVM_SEED = 1234

In [11]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL'])

In [12]:
classifications = sections
classifications_type = "sections"
VALIDATION_METRICS_FILENAME= '{}_validation_metrics.pkl'.format(classifications_type)

In [13]:
class OneHotEncoder():
    
    def __init__(self, classifications):
        self.classifications = classifications
        self.one_hot_indices = {}

        # convert character classifications to bit vectors
        for i, clssf in enumerate(classifications):
            bits = [0] * len(classifications)
            bits[i] = 1
            self.one_hot_indices[clssf] = i
    
    def get_label_vector(self, labels):
        """
        classes: array of string with the classes assigned to the instance
        """
        output_vector = [0] * len(self.classifications)
        for label in labels:
            index = self.one_hot_indices[label]
            output_vector[index] = 1
            
        return output_vector

    
def get_training_data(doc2vec_model, classifications):
    one_hot_encoder = OneHotEncoder(classifications)
    training_data = []
    training_labels = []
    for doc_id in training_docs_list:
        # converting from memmap to a normal array
        normal_array = []
        normal_array[:] = doc2vec_model.docvecs[doc_id][:]
        training_data.append(normal_array)
        eligible_classifications = [clssf for clssf in doc_classification_map[doc_id] if clssf in classifications]
        training_labels.append(one_hot_encoder.get_label_vector(eligible_classifications))
    training_labels = np.array(training_labels)
    return training_data, training_labels

def get_validation_docs_with_inference_new(doc2vec_model, doc_classification_map, classifications, 
                                           val_docs_list, val_preprocessed_files_prefix, 
                                           val_preprocessed_docids_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation documents
    """

    def infer_one_doc(doc_tuple):
        #doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    one_hot_encoder = OneHotEncoder(classifications)
    if os.path.exists(os.path.join(GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)):
        info("===== Loading validation vectors")
        validation_labels = []
        validation_vectors_matrix = pickle.load(open(os.path.join(GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)))
        for validation_doc_id in val_docs_list:
            val_labels = [classf for classf in doc_classification_map[validation_doc_id] if classf in classifications]
            validation_labels.append(one_hot_encoder.get_label_vector(val_labels))
        validation_labels = np.array(validation_labels)
    else:
        validation_documents_reps = {}
        validation_vectors = []
        validation_labels = []
        info("===== Getting validation vectors with inference")

        # Multi-threaded inference
        validation_docs_iterator = DocumentBatchGenerator(validation_preprocessed_files_prefix, 
                                                          validation_preprocessed_docids_files_prefix, batch_size=None)
        generator_func = validation_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        validation_documents_reps = {}
        mini_batch_size = 1000
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {}".format(str(validation_docs_iterator.curr_index)))
            if threaded_reps_partial:
                #threaded_reps.extend(threaded_reps_partial)
                validation_documents_reps.update(threaded_reps_partial)
            else:
                break
                
        # create matrix for the validation vectors
        for validation_doc_id in val_docs_list:
            validation_vectors.append(validation_documents_reps[validation_doc_id])
            val_labels = [classf for classf in doc_classification_map[validation_doc_id] if classf in classifications]
            validation_labels.append(one_hot_encoder.get_label_vector(val_labels))
        validation_vectors_matrix = np.array(validation_vectors)
        validation_labels = np.array(validation_labels)
        pickle.dump(validation_vectors_matrix, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX), 'w'))
    
    return validation_vectors_matrix, validation_labels

In [32]:
ee = 'svm_iter_100_reg_0.001_classweights_balanced'.split('_')
SVM_ITER = ee[2]
SVM_REG = ee[4]
SVM_CLASSWEIGHTS = ee[6]

In [33]:
print SVM_ITER
print SVM_REG
print SVM_CLASSWEIGHTS

100
0.001
balanced


In [14]:
finished_doc2vec_methods = ['doc2vec_size_100_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None',
                           'doc2vec_size_200_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None',
                           'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None']

In [17]:
for doc2vec_method in natural_sort(get_subdirectories(ROOT_RESULTS_LOCATION)):
    if doc2vec_method in finished_doc2vec_methods: 
        info('skipping {}'.format(doc2vec_method))
        continue
        
    print '********************* {}'.format(doc2vec_method)
    epochs_path = os.path.join(ROOT_RESULTS_LOCATION, doc2vec_method)
    # this will have the structure dict of classifiers -> dict of metrics -> list of values throughout the epoch
    for epoch in natural_sort(get_subdirectories(epochs_path)):
        print epoch
        epoch_path =  os.path.join(epochs_path, epoch)
        
        if os.path.exists(os.path.join(epoch_path, MODEL_PREFIX)):
            doc2vec_model = Doc2Vec.load(os.path.join(epoch_path, MODEL_PREFIX))
        else:
            info('No Doc2vec model found for {}. Exiting..'.format(epoch))
            break
            
        GLOBAL_VARS.MODEL_NAME = epoch_path

        # Validation Metrics
        info('Getting Validation Embeddings')
        Xv, yv = get_validation_docs_with_inference_new(doc2vec_model, doc_classification_map, classifications, 
                                                        validation_docs_list, validation_preprocessed_files_prefix,
                                                        validation_preprocessed_docids_files_prefix)

        for classifier in get_subdirectories(epoch_path):
            print classifier
            
            classifier_path = os.path.join(epoch_path, classifier)
            if(os.path.exists(os.path.join(classifier_path, CLASSIFIER))):
                info('Loading Classifier')
                clf = pickle.load(open(os.path.join(classifier_path, CLASSIFIER), 'r'))
            else:
                
                info('No Classifier found for {} in {}'.format(classifier, epoch))
                info('Getting training Data')
                X, y = get_training_data(doc2vec_model, classifications)
                info('Training Classifier')
                classifier_parts = classifier.split('_')
                SVM_ITERATIONS = classifier[2]
                SVM_REG = classifier[4]
                SVM_CLASS_WEIGHTS = classifier[6] if classifier[6] != 'None' else None
                clf = OneVsRestClassifier(linear_model.SGDClassifier(loss='hinge', penalty='l2', 
                                                                     #alpha is the 1/C parameter
                                                                     alpha=SVM_REG, fit_intercept=True, n_iter=SVM_ITERATIONS,
                                                                     #n_jobs=-1 means use all cpus
                                                                     shuffle=True, verbose=0, n_jobs=1,
                                                                     #eta0 is the learning rate when we use constant configuration
                                                                     random_state=SVM_SEED, learning_rate='optimal', eta0=0.0, 
                                                                     class_weight=SVM_CLASS_WEIGHTS, warm_start=False), n_jobs=1)

                # Training of a classifier
                clf.fit(X,y)
                pickle.dump(clf, open(os.path.join(classifier_path, CLASSIFIER), 'w'))

            info('Evaluating on Validation Data')
            yvp = clf.predict(Xv)
            yvp_score = clf.decision_function(Xv)
            print yvp
            info('Getting Validation Metrics')
            validation_metrics = get_metrics(yv, yvp_score, yvp)
            print "** Validation Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
                validation_metrics['coverage_error'], validation_metrics['average_num_of_labels'], 
                validation_metrics['top_1'], validation_metrics['top_3'], validation_metrics['top_5'], 
                validation_metrics['f1_micro'], validation_metrics['f1_macro'], validation_metrics['total_positive'])
            
            pickle.dump(validation_metrics, open(os.path.join(classifier_path, VALIDATION_METRICS_FILENAME), 'w'))


2017-01-17 20:15:38,512 : INFO : loading Doc2Vec object from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model


********************* doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None
epoch_1


2017-01-17 20:15:45,346 : INFO : loading docvecs recursively from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model.docvecs.* with mmap=None
2017-01-17 20:15:45,347 : INFO : loading doctag_syn0 from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model.docvecs.doctag_syn0.npy with mmap=None
2017-01-17 20:15:47,038 : INFO : loading syn1neg from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model.syn1neg.npy with mmap=None
2017-01-17 20:15:52,756 : INFO : loading syn0 from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model.syn0.npy with mmap=None
2017-01-17 20:15:53,

svm_iter_10_reg_0.001_classweights_balanced


2017-01-17 20:16:19,129 : INFO : Getting Validation Metrics


...

** Validation Metrics: Cov Err: 3.659, Avg Labels: 1.150, 
		 Top 1: 0.296, Top 3: 0.578, Top 5: 0.778, 
		 F1 Micro: 0.276, F1 Macro: 0.185, Total Pos: 502,399
epoch_2


2017-01-17 20:57:39,866 : INFO : loading docvecs recursively from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_2/model.docvecs.* with mmap=None
2017-01-17 20:57:39,867 : INFO : loading doctag_syn0 from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_2/model.docvecs.doctag_syn0.npy with mmap=None
2017-01-17 20:57:43,629 : INFO : loading syn1neg from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_2/model.syn1neg.npy with mmap=None
2017-01-17 20:58:04,753 : INFO : loading syn0 from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_2/model.syn0.npy with mmap=None
2017-01-17 20:58:05,

svm_iter_100_reg_0.01_classweights_balanced


2017-01-17 20:58:43,732 : INFO : Getting Validation Metrics


[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]
 [0 1 0 ..., 0 0 1]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]]
** Validation Metrics: Cov Err: 3.559, Avg Labels: 1.150, 
		 Top 1: 0.299, Top 3: 0.611, Top 5: 0.790, 
		 F1 Micro: 0.272, F1 Macro: 0.161, Total Pos: 430,943
********************* vocab_model
